# QUBO Formulation for Sudoku Puzzles

**Author:** Jonah Minkoff  
**Date:** 2025

This notebook demonstrates how to formulate Sudoku puzzles as Quadratic Unconstrained Binary Optimization (QUBO) problems, suitable for quantum annealing or classical optimization solvers.

## 1. Problem Definition

### The Sudoku Problem

Sudoku is a constraint satisfaction puzzle where we must fill an N×N grid with digits 1 through N such that:

1. **Each cell** contains exactly one digit
2. **Each row** contains each digit exactly once
3. **Each column** contains each digit exactly once
4. **Each box** contains each digit exactly once

For a 4×4 Sudoku:
- 4 rows, 4 columns
- 4 boxes of size 2×2
- Digits 1, 2, 3, 4

For a 9×9 Sudoku:
- 9 rows, 9 columns
- 9 boxes of size 3×3
- Digits 1 through 9

Some cells may be given (clues), reducing the problem's complexity.

## 2. Mathematical Formulation

### Binary Variables

We encode the problem using binary variables with **one-hot encoding**:

$$x_{i,j,k} = \begin{cases} 
1 & \text{if cell }(i,j)\text{ contains digit }k+1 \\
0 & \text{otherwise}
\end{cases}$$

Where:
- $i \in \{0, 1, ..., N-1\}$ is the row index
- $j \in \{0, 1, ..., N-1\}$ is the column index
- $k \in \{0, 1, ..., N-1\}$ is the digit index (0 represents digit 1, etc.)

For a 4×4 Sudoku: **64 binary variables**  
For a 9×9 Sudoku: **729 binary variables**

### Energy Function

The total energy is:

$$E = E_1 + E_2 + E_3 + E_4$$

Where:
- $E_1$: Each cell has exactly one digit
- $E_2$: Each row has each digit exactly once
- $E_3$: Each column has each digit exactly once
- $E_4$: Each box has each digit exactly once

Each constraint is formulated as a squared penalty:

$$E_1 = \sum_{i,j} \left[\sum_k x_{i,j,k} - 1\right]^2$$

$$E_2 = \sum_{i,k} \left[\sum_j x_{i,j,k} - 1\right]^2$$

$$E_3 = \sum_{j,k} \left[\sum_i x_{i,j,k} - 1\right]^2$$

$$E_4 = \sum_{\text{box},k} \left[\sum_{(i,j) \in \text{box}} x_{i,j,k} - 1\right]^2$$

A valid solution has $E = 0$ (all constraints satisfied).

## 3. QUBO Construction

### Setup and Imports

We'll use our pre-tested functions for QUBO construction from the accompanying Python modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import our tested QUBO construction functions
import sys
sys.path.append('problem_formation_and_evaluation/energy_calc')
sys.path.append('problem_formation_and_evaluation/QUBO_construction')

from calc_mods import total_energy, is_valid_solution, tensor_to_grid, print_grid
from qubo_generation import (
    build_E1, build_E2, build_E3, build_E4,
    build_sudoku_qubo, evaluate_qubo, print_qubo_stats
)

### Helper Functions for Visualization

In [ ]:
def visualize_qubo_matrix(Q, title="QUBO Matrix", show_values=False):
    """
    Visualize a QUBO matrix as a heatmap.
    
    Args:
        Q: QUBO matrix
        title: Title for the plot
        show_values: Whether to show numerical values in cells
    """
    plt.figure(figsize=(10, 8))
    
    if show_values and Q.shape[0] <= 20:
        sns.heatmap(Q, annot=True, fmt='.1f', cmap='RdBu_r', center=0,
                   square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    else:
        sns.heatmap(Q, cmap='RdBu_r', center=0,
                   square=True, cbar_kws={"shrink": 0.8})
    
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Variable Index', fontsize=11)
    plt.ylabel('Variable Index', fontsize=11)
    plt.tight_layout()
    plt.show()


def print_polynomial(poly_terms, constant, title, max_terms=10):
    """
    Print a polynomial in readable format.
    
    Args:
        poly_terms: List of polynomial term strings
        constant: Constant term
        title: Title for the polynomial
        max_terms: Maximum number of terms to display
    """
    print(f"\n{title}")
    print("=" * 70)
    print(f"Number of terms: {len(poly_terms)}")
    print(f"Constant: {constant}")
    
    if len(poly_terms) > 0:
        print(f"\nFirst {min(max_terms, len(poly_terms))} terms:")
        for i, term in enumerate(poly_terms[:max_terms]):
            print(f"  {term}")
        
        if len(poly_terms) > max_terms:
            print(f"  ... ({len(poly_terms) - max_terms} more terms)")


def visualize_sudoku(grid, N, box_size, title="Sudoku Grid"):
    """
    Visualize a Sudoku grid.
    
    Args:
        grid: N×N numpy array with digit values
        N: Size of Sudoku
        box_size: Size of each box
        title: Title for the plot
    """
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Create colored grid
    ax.imshow(np.ones((N, N)), cmap='Greys', alpha=0.1)
    
    # Draw grid lines
    for i in range(N + 1):
        linewidth = 2 if i % box_size == 0 else 0.5
        ax.axhline(i - 0.5, color='black', linewidth=linewidth)
        ax.axvline(i - 0.5, color='black', linewidth=linewidth)
    
    # Add numbers
    for i in range(N):
        for j in range(N):
            if grid[i, j] != 0:
                ax.text(j, i, str(grid[i, j]), 
                       ha='center', va='center', fontsize=16, fontweight='bold')
    
    ax.set_xlim(-0.5, N - 0.5)
    ax.set_ylim(N - 0.5, -0.5)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 4. Example: 4×4 Sudoku Puzzle

Let's work through a complete example with a partially filled 4×4 Sudoku puzzle.

**Given cells (clues):**
```
2 _ | 4 _
_ 3 | _ 1
-----+----
1 _ | 3 _
_ 4 | _ 2
```

**Solution:**
```
2 1 | 4 3
4 3 | 2 1
-----+----
1 2 | 3 4
3 4 | 1 2
```

In [ ]:
# Define the 4×4 Sudoku puzzle
N = 4
box_size = 2

# Given cells (using 0-indexed positions, 1-indexed digits)
givens = {
    (0, 0): 2,           (0, 2): 4,
              (1, 1): 3,           (1, 3): 1,
    (2, 0): 1,           (2, 2): 3,
              (3, 1): 4,           (3, 3): 2
}

print(f"Sudoku size: {N}×{N}")
print(f"Box size: {box_size}×{box_size}")
print(f"Number of givens: {len(givens)}")
print(f"Number of variables: {N * N * N} (before eliminating givens)")
print(f"Free variables: {(N*N - len(givens)) * N}")

# Visualize the puzzle
print("\nGiven cells:")
for i in range(N):
    row_str = ""
    for j in range(N):
        if j > 0 and j % box_size == 0:
            row_str += "| "
        if (i, j) in givens:
            row_str += str(givens[(i, j)]) + " "
        else:
            row_str += "_ "
    print(row_str.rstrip())
    if i == box_size - 1:
        print("-" * (2*N + box_size - 1))

## 4a. Building QUBO Components

We'll build each energy term separately using our tested functions.

### E1: Each Cell Has Exactly One Digit

This ensures every cell contains **exactly one digit** (no empty cells, no multiple digits).

$$E_1 = \sum_{i,j} \left[\sum_k x_{i,j,k} - 1\right]^2$$

When expanded for QUBO:
$$E_1 = \sum_{i,j} \left[-\sum_k x_{i,j,k} + 2\sum_{k<k'} x_{i,j,k} \cdot x_{i,j,k'} + 1\right]$$

In [ ]:
# Build E1 (each cell has exactly one digit)
Q_E1, poly_E1, const_E1 = build_E1(N, givens)

print("=" * 70)
print("E1: Each Cell Has Exactly One Digit")
print("=" * 70)
print(f"Number of terms: {len(poly_E1)}")
print(f"Constant: {const_E1}")
print(f"\nNote: {len(givens)} cells are given, so they don't contribute variables")

In [ ]:
# View E1 polynomial
print_polynomial(poly_E1, const_E1, "E1: Each Cell Has Exactly One Digit", max_terms=15)

### E2: Each Row Has Each Digit Exactly Once

This ensures no row has duplicate or missing digits.

$$E_2 = \sum_{i,k} \left[\sum_j x_{i,j,k} - 1\right]^2$$

When some cells are given, the constraint adjusts to account for already-placed digits.

In [ ]:
# Build E2 (each row has each digit exactly once)
Q_E2, poly_E2, const_E2 = build_E2(N, givens)

print("=" * 70)
print("E2: Each Row Has Each Digit Exactly Once")
print("=" * 70)
print(f"Number of terms: {len(poly_E2)}")
print(f"Constant: {const_E2}")

In [ ]:
# View E2 polynomial
print_polynomial(poly_E2, const_E2, "E2: Each Row Has Each Digit", max_terms=15)

### E3: Each Column Has Each Digit Exactly Once

Similar to row constraints, but applied to columns.

$$E_3 = \sum_{j,k} \left[\sum_i x_{i,j,k} - 1\right]^2$$

In [ ]:
# Build E3 (each column has each digit exactly once)
Q_E3, poly_E3, const_E3 = build_E3(N, givens)

print("=" * 70)
print("E3: Each Column Has Each Digit Exactly Once")
print("=" * 70)
print(f"Number of terms: {len(poly_E3)}")
print(f"Constant: {const_E3}")

In [ ]:
# View E3 polynomial
print_polynomial(poly_E3, const_E3, "E3: Each Column Has Each Digit", max_terms=15)

### E4: Each Box Has Each Digit Exactly Once

For a 4×4 Sudoku, we have four 2×2 boxes that must each contain digits 1-4 exactly once.

$$E_4 = \sum_{\text{box},k} \left[\sum_{(i,j) \in \text{box}} x_{i,j,k} - 1\right]^2$$

In [ ]:
# Build E4 (each box has each digit exactly once)
Q_E4, poly_E4, const_E4 = build_E4(N, box_size, givens)

print("=" * 70)
print("E4: Each Box Has Each Digit Exactly Once")
print("=" * 70)
print(f"Number of terms: {len(poly_E4)}")
print(f"Constant: {const_E4}")

In [ ]:
# View E4 polynomial
print_polynomial(poly_E4, const_E4, "E4: Each Box Has Each Digit", max_terms=15)

### Combining All Terms

Now we combine all four energy components into a single QUBO matrix. Since this is a pure constraint satisfaction problem (no objective to minimize), all Lagrange multipliers are set to 1.

In [ ]:
# Combine all QUBO matrices (all Lagrange multipliers = 1)
Q_total = Q_E1 + Q_E2 + Q_E3 + Q_E4
offset_total = const_E1 + const_E2 + const_E3 + const_E4

print("=" * 70)
print("Combined QUBO Matrix")
print("=" * 70)
print_qubo_stats(Q_total, N, givens)
print(f"Total constant offset: {offset_total}")

# Visualize the combined QUBO
visualize_qubo_matrix(Q_total, f"Combined QUBO Matrix ({N}×{N} Sudoku with Givens)", show_values=False)

## 5. Validation and Testing

### Test Case 1: Correct Solution

Let's test the correct solution to the puzzle.

In [ ]:
# Correct solution bitstring
bitstring_correct = (
    "0100" + "1000" + "0001" + "0010" +  # Row 0: 2,1,4,3
    "0001" + "0010" + "0100" + "1000" +  # Row 1: 4,3,2,1
    "1000" + "0100" + "0010" + "0001" +  # Row 2: 1,2,3,4
    "0010" + "0001" + "1000" + "0100"    # Row 3: 3,4,1,2
)

print("Test Case 1: Correct Solution")
print("=" * 70)

# Evaluate using calc_mods
energy_calc, breakdown = total_energy(bitstring_correct, N, box_size, verbose=False)
print(f"\ncalc_mods energy: {energy_calc}")
print(f"Breakdown: E1={breakdown[0]}, E2={breakdown[1]}, E3={breakdown[2]}, E4={breakdown[3]}")
print(f"Valid solution: {is_valid_solution(breakdown)}")

# Evaluate using QUBO
Q_full, var_to_idx, idx_to_var, offset_full = build_sudoku_qubo(N, box_size, givens)
energy_qubo = evaluate_qubo(Q_full, bitstring_correct, offset_full)
print(f"\nQUBO energy: {energy_qubo}")

# Visualize the solution
from calc_mods import bitstring_to_tensor
x = bitstring_to_tensor(bitstring_correct, N)
grid = tensor_to_grid(x, N)
print("\nSolution grid:")
print_grid(grid, N, box_size)

### Test Case 2: Incorrect Solution

Now let's test an incorrect solution that violates constraints.

In [ ]:
# Incorrect solution - swap cells (0,1) and (0,3): 1↔3
bitstring_wrong = (
    "0100" + "0010" + "0001" + "1000" +  # Row 0: 2,3,4,1 (wrong!)
    "0001" + "0010" + "0100" + "1000" +  # Row 1: 4,3,2,1
    "1000" + "0100" + "0010" + "0001" +  # Row 2: 1,2,3,4
    "0010" + "0001" + "1000" + "0100"    # Row 3: 3,4,1,2
)

print("Test Case 2: Incorrect Solution")
print("=" * 70)

# Evaluate using calc_mods
energy_calc_wrong, breakdown_wrong = total_energy(bitstring_wrong, N, box_size, verbose=True)
print(f"\nValid solution: {is_valid_solution(breakdown_wrong)}")

# Evaluate using QUBO
energy_qubo_wrong = evaluate_qubo(Q_full, bitstring_wrong, offset_full)
print(f"\nQUBO energy: {energy_qubo_wrong}")
print(f"Energy difference from correct: {energy_qubo_wrong - energy_qubo}")

print("\nThe incorrect solution has HIGHER energy, as expected!")

## 6. Blank 4×4 Sudoku (No Givens)

Let's examine the QUBO for a completely blank 4×4 Sudoku to see the full problem complexity.

In [ ]:
# Build QUBO for blank 4×4 Sudoku
N_blank = 4
box_size_blank = 2
givens_blank = None

Q_blank, var_to_idx_blank, idx_to_var_blank, offset_blank = build_sudoku_qubo(
    N_blank, box_size_blank, givens_blank
)

print("=" * 70)
print("Blank 4×4 Sudoku (No Givens)")
print("=" * 70)
print_qubo_stats(Q_blank, N_blank, givens_blank)
print(f"Total constant offset: {offset_blank}")

# Visualize
visualize_qubo_matrix(Q_blank, "Blank 4×4 Sudoku QUBO Matrix", show_values=False)

In [ ]:
# Test the same solution on blank puzzle
energy_blank = evaluate_qubo(Q_blank, bitstring_correct, offset_blank)

print(f"Energy of correct solution on blank puzzle: {energy_blank}")
print("Energy is 0, confirming all constraints are satisfied!")

## 7. Scaling to 9×9 Sudoku

Let's demonstrate the scalability by examining the QUBO construction for a full 9×9 Sudoku (we won't solve it in this notebook due to the 729 variables).

In [ ]:
# Build QUBO for blank 9×9 Sudoku (construction only)
N_large = 9
box_size_large = 3
givens_large = None

print("Building 9×9 Sudoku QUBO...")
Q_large, var_to_idx_large, idx_to_var_large, offset_large = build_sudoku_qubo(
    N_large, box_size_large, givens_large
)

print("\n" + "=" * 70)
print("Blank 9×9 Sudoku")
print("=" * 70)
print_qubo_stats(Q_large, N_large, givens_large)
print(f"Total constant offset: {offset_large}")

print("\nThe 9×9 QUBO has been successfully constructed!")
print(f"Matrix size: {Q_large.shape[0]}×{Q_large.shape[1]}")

## 8. Analysis and Discussion

### Key Observations

1. **Matrix Structure**:
   - The QUBO matrix is very sparse (>95% zeros)
   - Most non-zero entries are on the diagonal (linear terms)
   - Off-diagonal entries represent interactions between variables

2. **Scaling**:
   - **4×4 Sudoku**: 64 variables, manageable on current quantum hardware
   - **9×9 Sudoku**: 729 variables, challenging but possible on larger QPUs
   - Givens significantly reduce the effective problem size

3. **Constraint Satisfaction**:
   - This is a pure constraint satisfaction problem (CSP)
   - Valid solutions have energy = 0
   - Invalid solutions have energy > 0 (proportional to violations)

4. **One-Hot Encoding**:
   - Simple and intuitive
   - Each constraint is easy to formulate
   - BUT: Uses many qubits (N³ for N×N Sudoku)
   - Alternative: Binary encoding uses ~50% fewer qubits but more complex constraints

In [ ]:
# Analyze scaling
sizes = [(4, 4), (9, 9), (16, 16), (25, 25)]
data = []

for n in [4, 9, 16, 25]:
    n_vars = n ** 3
    n_constraints = 4 * n * n
    # Estimate non-zero entries (empirical from our examples)
    sparsity = 1 - (n_vars + 4 * 6 * n * n) / (n_vars ** 2)
    data.append({
        'N': n,
        'Variables': n_vars,
        'Constraints': n_constraints,
        'Matrix Size': f'{n_vars}×{n_vars}',
        'Sparsity': f'{sparsity*100:.2f}%'
    })

import pandas as pd
df = pd.DataFrame(data)
print("\nScaling Analysis:")
print("=" * 70)
print(df.to_string(index=False))

# Plot scaling
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ns = [4, 9, 16, 25]
vars = [n**3 for n in ns]
constraints = [4*n*n for n in ns]

ax1.plot(ns, vars, 'o-', linewidth=2, markersize=8, label='Variables')
ax1.plot(ns, constraints, 's-', linewidth=2, markersize=8, label='Constraints')
ax1.set_xlabel('Sudoku Size (N×N)', fontsize=11)
ax1.set_ylabel('Count', fontsize=11)
ax1.set_title('Scaling of Variables and Constraints', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.semilogy(ns, vars, 'o-', linewidth=2, markersize=8)
ax2.set_xlabel('Sudoku Size (N×N)', fontsize=11)
ax2.set_ylabel('Number of Variables (log scale)', fontsize=11)
ax2.set_title('Variable Growth (Logarithmic Scale)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Solving the QUBO

This QUBO formulation can be solved using:

1. **Quantum Annealing** (e.g., D-Wave Systems)
   - Native QUBO format
   - Can handle sparse matrices efficiently
   - Limited by qubit count (5000+ qubits on Advantage)

2. **Gate-Based Quantum** (e.g., IBM, Google)
   - Requires conversion to QAOA or VQE
   - Currently limited to smaller problems

3. **Classical Solvers**
   - Simulated annealing
   - Tabu search
   - Integer programming solvers (after conversion)
   - Often very effective for Sudoku due to CSP structure

4. **Hybrid Approaches**
   - Quantum-classical hybrid algorithms
   - Decomposition methods

## Conclusion

This notebook demonstrated:

1. **Problem Formulation**: How to encode Sudoku as a QUBO problem using one-hot encoding
2. **Constraint Construction**: Four energy components (E1, E2, E3, E4) ensuring valid solutions
3. **Validation**: Testing with correct and incorrect solutions
4. **Scalability**: Analysis from 4×4 to 25×25 Sudoku

### Key Takeaways

- **One-hot encoding** is intuitive but qubit-intensive (N³ variables)
- The problem naturally decomposes into **four independent constraint types**
- **Givens reduce complexity** significantly by eliminating variables
- **9×9 Sudoku** (729 qubits) is at the edge of current quantum hardware capabilities
- Alternative encodings (binary, domain wall) could reduce qubit requirements

### Future Directions

1. Implement **binary encoding** to reduce qubit count by ~50%
2. Test on actual **quantum hardware** (D-Wave, IBM)
3. Compare **classical vs quantum** solving times
4. Explore **hybrid decomposition** approaches for larger puzzles

---

*For implementation details, see the accompanying Python modules:*
- `calc_mods.py`: Direct energy calculation
- `qubo_generation.py`: QUBO matrix construction
- `construction_test.py`: Validation test suite